In [58]:
from pathlib import Path
import pandas as pd

In [59]:
csv_file_dir = Path("../../content/tables/TRP/")
params_to_keep = set(
    ["CRRA", "BeqFac", "BeqShift", "WealthShare", "WealthShift", "criterion"]
)

# Get all files in the directory
files = [f for f in csv_file_dir.iterdir() if f.is_file()]

parameters = []

# Iterate over each file
for file in files:
    file_name = file.stem.replace("_estimate_results", "")
    # Read the CSV file and convert it to a dictionary
    res = pd.read_csv(file, header=None)
    res = res.set_index(res.columns[0])[res.columns[1]].to_dict()

    # Create a new dictionary for this file
    file_parameters = {}

    labor = 0
    stock = 0

    if "Sub" in file_name:
        if "(Labor)" in file_name:
            labor = 1
            file_name = file_name.replace("(Labor)", "")

        if "(Stock)" in file_name:
            stock = 1
            file_name = file_name.replace("(Stock)", "")

    file_name = file_name.replace("Sub", "").replace("Market", "")
    file_parameters["Name"] = file_name
    file_parameters["Stock"] = stock

    # Iterate over each parameter we want to keep
    for param in params_to_keep:
        # If the parameter is in the dictionary, add it to the file's parameters
        if param in res:
            file_parameters[param] = res[param]

    # Add the file's parameters to the overall parameters dictionary
    parameters.append(file_parameters)

In [60]:
def format_df(df):
    for col in df.columns:
        # Check if column is of float type
        if col in params_to_keep:
            df[col] = df[col].astype(float).round(3).fillna("")
            pass
        # Check if column contains only 0 and 1
        else:
            df[col] = df[col].map({0: "", 1: "✔️"})
    return df


df = pd.DataFrame(parameters).set_index("Name")
format_df(df).sort_index()

,Stock,BeqShift,CRRA,criterion,BeqFac,WealthShift,WealthShare
Name,,,,,,,
IndShock,✔️,,4.385,494.161,,,
IndShock,,,4.476,509.235,,,
Portfolio,✔️,,2.535,2.428,,,
Portfolio,,,8.341,0.883,,,
WarmGlow,✔️,13.52,4.375,494.114,37.337,,
WarmGlow,,0.003,2.345,404.255,34.655,,
WarmGlowPortfolio,,21.041,6.154,0.524,39.623,,
WarmGlowPortfolio,✔️,61.559,2.537,2.443,35.843,,
WealthPortfolio,✔️,,2.535,2.428,,50.944,0.0
